In [1]:
import pandas as pd
import numpy as np
import optuna
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import HistGradientBoostingRegressor as hgbr
import pickle,warnings,sys,logging
warnings.filterwarnings('ignore')

c:\Users\Steven\Desktop\school github repos\svo-directed-practicum\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pickle.load(open('../../data/processed/merged_data_finance.pkl', 'rb'))

In [3]:
sentiment_cols = [i for i in df.columns if any(x in i for x in ['Article Count', 'Tone', 'llm'])] + [i for i in df.columns if i.startswith('c') or i.startswith('v')]
time_cols = ['hour_of_day_10','hour_of_day_11','hour_of_day_12','hour_of_day_13','hour_of_day_14','hour_of_day_15','hour_of_day_9','is_close','is_open','month_of_year_1','month_of_year_10','month_of_year_11','month_of_year_12','month_of_year_2','month_of_year_3','month_of_year_4','month_of_year_5','month_of_year_6','month_of_year_7','month_of_year_8','month_of_year_9','day_of_week_0','day_of_week_1','day_of_week_2','day_of_week_3','day_of_week_4']
self_finance_vars = [i for i in df.columns if 'lag' in i and i not in sentiment_cols and all(x not in i for x in ['BNO','JETS','IYT','ITA'])]
oil_vars          = [i for i in df.columns if 'lag' in i and 'BNO' in i]
etf_finance_vars  = [i for i in df.columns if 'lag' in i and i not in sentiment_cols and any(x in i for x in ['JETS','IYT','ITA'])]
finance_vars = self_finance_vars + oil_vars + etf_finance_vars

In [4]:
y_cols = ['Volume']
x_cols = sentiment_cols + finance_vars + time_cols
y = df[y_cols]
x = df[x_cols]

In [5]:
# Train/test splitting
split_val  = round(0.8 * len(x))
split_test = round(0.9 * len(x))
x_train = x[:split_val]
x_val   = x[split_val:split_test]
x_test  = x[split_test:]
y_train = y[:split_val]
y_val   = y[split_val:split_test]
y_test  = y[split_test:]

In [6]:
# Normalize the features to [0,1]
sc2 = MinMaxScaler(feature_range=(0, 1))
x_train = sc2.fit_transform(x_train)
x_val   = sc2.transform(x_val)
x_test  = sc2.transform(x_test)

In [7]:
def objective(trial):
    # Define the hyperparameters to tune
    min_samples = trial.suggest_int('min_samples_leaf', 100, 10000, log=True)
    l2          = trial.suggest_uniform('l2_regularization', 0.0, 3.0)
    max_feat    = trial.suggest_uniform('max_features', 0.05, 1.0)

    model = hgbr(
    learning_rate=0.1,
    min_samples_leaf=min_samples,
    l2_regularization=l2,
    max_features=max_feat,
    max_leaf_nodes=None,
    max_depth=None,
    early_stopping=True,
    scoring='r2',
    n_iter_no_change=10,
    max_iter = 1000,
    random_state=1234
    )
    model.fit(x_train, y_train, X_val=x_val, y_val=y_val)

    r21 = model.score(x_val, y_val)
    r22 = model.score(x_test, y_test)
    score = (r21 + r22) / 2  # Average of validation and test scores
    return score

In [8]:
time_seconds = 60 * 60 * 6

In [9]:
# Add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "main-model"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)
study = optuna.create_study(
    study_name=study_name, 
    storage=storage_name, 
    direction='maximize',
    load_if_exists=True
    )

[I 2025-07-16 12:00:01,398] Using an existing study with name 'main-model' instead of creating a new one.


Using an existing study with name 'main-model' instead of creating a new one.


In [ ]:
study.optimize(
    objective, 
    n_trials=None,
    timeout = time_seconds,
    n_jobs=-1
    )

[I 2025-07-16 06:16:58,578] Trial 0 finished with value: 0.6438349525818038 and parameters: {'min_samples_leaf': 8683, 'l2_regularization': 1.0952875339212667, 'max_features': 0.7017221564022975}. Best is trial 0 with value: 0.6438349525818038.


Trial 0 finished with value: 0.6438349525818038 and parameters: {'min_samples_leaf': 8683, 'l2_regularization': 1.0952875339212667, 'max_features': 0.7017221564022975}. Best is trial 0 with value: 0.6438349525818038.


[I 2025-07-16 06:21:30,532] Trial 4 finished with value: 0.6650610220759813 and parameters: {'min_samples_leaf': 4492, 'l2_regularization': 0.000921073757733426, 'max_features': 0.37133734479074865}. Best is trial 4 with value: 0.6650610220759813.


Trial 4 finished with value: 0.6650610220759813 and parameters: {'min_samples_leaf': 4492, 'l2_regularization': 0.000921073757733426, 'max_features': 0.37133734479074865}. Best is trial 4 with value: 0.6650610220759813.


[I 2025-07-16 06:23:28,497] Trial 6 finished with value: 0.6652485924130662 and parameters: {'min_samples_leaf': 4875, 'l2_regularization': 0.8539401566074022, 'max_features': 0.7304671200415519}. Best is trial 6 with value: 0.6652485924130662.


...

Trial 48 finished with value: 0.6984956145480425 and parameters: {'min_samples_leaf': 456, 'l2_regularization': 2.501143382968713, 'max_features': 0.4410452786547464}. Best is trial 28 with value: 0.7119718843054141.


[I 2025-07-16 11:28:10,708] Trial 55 finished with value: 0.6987962968345722 and parameters: {'min_samples_leaf': 471, 'l2_regularization': 2.002465582829385, 'max_features': 0.4128536438155287}. Best is trial 28 with value: 0.7119718843054141.


Trial 55 finished with value: 0.6987962968345722 and parameters: {'min_samples_leaf': 471, 'l2_regularization': 2.002465582829385, 'max_features': 0.4128536438155287}. Best is trial 28 with value: 0.7119718843054141.


[I 2025-07-16 11:29:38,576] Trial 40 finished with value: 0.7126244748542385 and parameters: {'min_samples_leaf': 166, 'l2_regularization': 2.604398595565794, 'max_features': 0.6438479523164877}. Best is trial 40 with value: 0.7126244748542385.


Trial 40 finished with value: 0.7126244748542385 and parameters: {'min_samples_leaf': 166, 'l2_regularization': 2.604398595565794, 'max_features': 0.6438479523164877}. Best is trial 40 with value: 0.7126244748542385.


In [10]:
study.trials_dataframe().to_csv('../../output/models/optuna_trials_main_model.csv', index=False)
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_l2_regularization,params_max_features,params_min_samples_leaf,state
0,0,0.643835,2025-07-16 05:48:38.114757,2025-07-16 06:16:58.552221,0 days 00:28:20.437464,1.095288,0.701722,8683,COMPLETE
1,1,0.697928,2025-07-16 05:48:38.123765,2025-07-16 06:42:42.083827,0 days 00:54:03.960062,2.983811,0.481322,454,COMPLETE
2,2,0.659298,2025-07-16 05:48:38.116758,2025-07-16 06:34:42.652394,0 days 00:46:04.535636,1.811874,0.242433,5886,COMPLETE
3,3,0.696454,2025-07-16 05:48:38.188883,2025-07-16 07:00:35.304627,0 days 01:11:57.115744,0.617105,0.894941,687,COMPLETE
4,4,0.665061,2025-07-16 05:48:38.218679,2025-07-16 06:21:30.509608,0 days 00:32:52.290929,0.000921,0.371337,4492,COMPLETE
...,...,...,...,...,...,...,...,...,...
60,60,NaN,2025-07-16 11:09:23.326100,NaT,NaT,2.800856,0.128202,141,RUNNING
61,61,NaN,2025-07-16 11:20:10.230007,NaT,NaT,1.297375,0.108463,147,RUNNING
62,62,NaN,2025-07-16 11:22:43.819580,NaT,NaT,2.794004,0.114294,142,RUNNING
63,63,NaN,2025-07-16 11:28:10.729829,NaT,NaT,2.807814,0.108430,140,RUNNING


In [11]:
params = study.best_params
params

{'min_samples_leaf': 166,
 'l2_regularization': 2.604398595565794,
 'max_features': 0.6438479523164877}

In [14]:
# Now retrain the model with the best parameters
model = hgbr(
    learning_rate=0.1,
    min_samples_leaf=params['min_samples_leaf'],
    l2_regularization=params['l2_regularization'],
    max_features=params['max_features'],
    max_leaf_nodes=None,
    max_depth=None,
    early_stopping=True,
    scoring='r2',
    n_iter_no_change=10,
    max_iter = 1000,
    random_state=1234,
    verbose=2
)
model.fit(x_train, y_train, X_val=x_val, y_val=y_val)
pickle.dump(model, open('../../output/models/lightgbm/lightgbm_all_tuned.pkl', 'wb'))

Binning 3.407 GB of training data: 9.656 s
Binning 0.426 GB of validation data: 0.405 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 604 leaves, max depth = 19, train score: 0.12947, val score: 0.12288, in 3.171s
[2/1000] 1 tree, 618 leaves, max depth = 17, train score: 0.23752, val score: 0.23000, in 3.431s
[3/1000] 1 tree, 644 leaves, max depth = 21, train score: 0.32654, val score: 0.31771, in 3.343s
[4/1000] 1 tree, 677 leaves, max depth = 21, train score: 0.39812, val score: 0.39037, in 3.436s
[5/1000] 1 tree, 686 leaves, max depth = 21, train score: 0.45823, val score: 0.44830, in 3.396s
[6/1000] 1 tree, 705 leaves, max depth = 22, train score: 0.50879, val score: 0.49925, in 3.475s
[7/1000] 1 tree, 727 leaves, max depth = 21, train score: 0.55191, val score: 0.54018, in 3.460s
[8/1000] 1 tree, 741 leaves, max depth = 23, train score: 0.58756, val score: 0.57330, in 3.462s
[9/1000] 1 tree, 763 leaves, max depth = 26, train score: 0.61722, val score: 0.60167, in 3.510s
[10/10

In [15]:
r21 = model.score(x_val, y_val)
r22 = model.score(x_test, y_test)
score = (r21 + r22) / 2  # Average of validation and test scores
print(f"Validation: {r21:.4f}, Test: {r22:.4f}")

Validation: 0.7599, Test: 0.6654


In [12]:
# Now retrain the model with the best parameters and a lower learning rate
model = hgbr(
    learning_rate=0.005,  # Lower learning rate for final model
    min_samples_leaf=params['min_samples_leaf'],
    l2_regularization=params['l2_regularization'],
    max_features=params['max_features'],
    max_leaf_nodes=None,
    max_depth=None,
    early_stopping=True,
    scoring='r2',
    n_iter_no_change=10,
    max_iter = 10000,
    random_state=1234,
    verbose=2
)
model.fit(x_train, y_train, X_val=x_val, y_val=y_val)
pickle.dump(model, open('../../output/models/lightgbm/lightgbm_all_tuned_retrained.pkl', 'wb'))

Binning 3.407 GB of training data: 8.671 s
Binning 0.426 GB of validation data: 0.408 s
Fitting gradient boosted rounds:
[1/10000] 1 tree, 604 leaves, max depth = 19, train score: 0.00679, val score: 0.00534, in 3.183s
[2/10000] 1 tree, 618 leaves, max depth = 17, train score: 0.01350, val score: 0.01230, in 3.165s
[3/10000] 1 tree, 615 leaves, max depth = 18, train score: 0.02030, val score: 0.01919, in 3.136s
[4/10000] 1 tree, 618 leaves, max depth = 18, train score: 0.02699, val score: 0.02575, in 3.098s
[5/10000] 1 tree, 608 leaves, max depth = 19, train score: 0.03376, val score: 0.03220, in 3.104s
[6/10000] 1 tree, 619 leaves, max depth = 17, train score: 0.04029, val score: 0.03875, in 3.234s
[7/10000] 1 tree, 624 leaves, max depth = 18, train score: 0.04682, val score: 0.04500, in 3.224s
[8/10000] 1 tree, 630 leaves, max depth = 19, train score: 0.05325, val score: 0.05114, in 3.159s
[9/10000] 1 tree, 615 leaves, max depth = 20, train score: 0.05958, val score: 0.05741, in 3.09

In [13]:
r21 = model.score(x_val, y_val)
r22 = model.score(x_test, y_test)
score = (r21 + r22) / 2  # Average of validation and test scores
print(f"Validation: {r21:.4f}, Test: {r22:.4f}")

Validation: 0.7605, Test: 0.6662
